# Guía avanzada de Apache Spark

In [59]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max, sum, udf
from pyspark.sql.types import IntegerType
import requests

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("COVID19 Analysis") \
    .getOrCreate()

# Descargar el archivo CSV
covid_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
response = requests.get(covid_url)
with open("/tmp/covid_data.csv", "wb") as f:
    f.write(response.content)

# Cargar el dataset
covid_df = spark.read.csv("/tmp/covid_data.csv", header=True, inferSchema=True)

# Obtener la última fecha disponible
date_columns = [c for c in covid_df.columns if c.count('/') == 2]
last_date = date_columns[-1]

# a) Mostrar los 10 países con más casos confirmados en la última fecha disponible
print("Top 10 países con más casos confirmados:")
covid_df.select("Country/Region", col(last_date).alias("Total_Cases")) \
    .groupBy("Country/Region") \
    .sum("Total_Cases") \
    .orderBy(col("sum(Total_Cases)").desc()) \
    .show(10)

# b) Calcular el total de casos globales para la última fecha disponible
total_global_cases = covid_df.select(sum(col(last_date))).first()[0]
print(f"Total de casos globales: {total_global_cases}")

# Detener la sesión de Spark
spark.stop()

24/09/04 03:42:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Top 10 países con más casos confirmados:
+--------------+----------------+
|Country/Region|sum(Total_Cases)|
+--------------+----------------+
|            US|       103802702|
|         India|        44690738|
|        France|        39866718|
|       Germany|        38249060|
|        Brazil|        37076053|
|         Japan|        33320438|
|  Korea, South|        30615522|
|         Italy|        25603510|
|United Kingdom|        24658705|
|        Russia|        22075858|
+--------------+----------------+
only showing top 10 rows

Total de casos globales: 676570149


In [60]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, explode, lit, sum as spark_sum, rank, dense_rank, udf, hour, unix_timestamp, date_format, lag, avg
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, IntegerType, DoubleType
import pandas as pd

spark = SparkSession.builder.appName("PySpark Exercises").getOrCreate()

# Descargar el archivo Parquet
taxi_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
response = requests.get(taxi_url)
with open("/tmp/taxi_data.parquet", "wb") as f:
    f.write(response.content)

# Cargar el dataset
taxi_df = spark.read.parquet("/tmp/taxi_data.parquet")

# a) Calcular el número total de viajes y el ingreso total para todo el mes
total_trips = taxi_df.count()
total_revenue = taxi_df.select(sum("total_amount")).first()[0]

print(f"Número total de viajes: {total_trips}")
print(f"Ingreso total: ${total_revenue:.2f}")

# b) Encontrar la hora del día con más viajes
busiest_hour = taxi_df.withColumn("hour", hour("tpep_pickup_datetime")) \
    .groupBy("hour") \
    .count() \
    .orderBy(col("count").desc()) \
    .first()

print(f"La hora con más viajes es: {busiest_hour['hour']}:00 con {busiest_hour['count']} viajes")

# c) Calcular la tarifa promedio por viaje
avg_fare = taxi_df.select(avg("total_amount")).first()[0]
print(f"La tarifa promedio por viaje es: ${avg_fare:.2f}")

24/09/04 03:42:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Número total de viajes: 3066766
Ingreso total: $82865192.22
La hora con más viajes es: 18:00 con 215889 viajes
La tarifa promedio por viaje es: $27.02


In [9]:
import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, udf, rank
from pyspark.sql.types import StringType
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import io
import logging

# Crear una sesión de Spark con configuración de log reducida
spark = SparkSession.builder \
    .appName("Wine Quality Analysis") \
    .getOrCreate()

# Reducir el nivel de log de Spark
spark.sparkContext.setLogLevel("ERROR")

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), sep=";")
df = spark.createDataFrame(data)

# Renombrar columnas para eliminar espacios
for column in df.columns:
    df = df.withColumnRenamed(column, column.replace(" ", "_"))

print(f"Total de muestras: {df.count()}")
df.groupBy("quality").count().orderBy("quality").show()

# Window Function: Ranking de vinos por alcohol
window_spec = Window.orderBy(col("alcohol").desc())
ranked_wines = df.withColumn("rank", rank().over(window_spec))
print("Top 5 vinos con mayor contenido de alcohol:")
ranked_wines.select("alcohol", "quality", "rank").filter(col("rank") <= 5).show()

# UDF: Categorizar calidad
@udf(returnType=StringType())
def categorize_quality(quality):
    return "Bajo" if quality <= 4 else "Medio" if quality <= 6 else "Alto"

df_with_category = df.withColumn("quality_category", categorize_quality(col("quality")))
print("Distribución por categoría de calidad:")
df_with_category.groupBy("quality_category").count().orderBy(col("count").desc()).show()

# Análisis de correlaciones
numeric_features = [c for c in df.columns if c != "quality"]
for feature in numeric_features:
    correlation = df.stat.corr(feature, "quality")
    print(f"Correlación entre {feature} y calidad: {correlation:.3f}")

# Visualización
plt.figure(figsize=(10, 6))
quality_counts = df.groupBy("quality").count().orderBy("quality").collect()
plt.bar([row['quality'] for row in quality_counts], [row['count'] for row in quality_counts])
plt.title("Distribución de la Calidad del Vino")
plt.xlabel("Calidad")
plt.ylabel("Número de Muestras")
plt.savefig("wine_quality_distribution.png")
plt.close()

print("""
Informe de Análisis de Calidad del Vino Tinto

1. Datos: Se utilizó el conjunto de datos de Calidad del Vino Tinto del repositorio UCI.

2. Hallazgos principales:
   a) La mayoría de los vinos tienen una calidad entre 5 y 6.
   b) Existe una correlación positiva entre el contenido de alcohol y la calidad.
   c) La acidez volátil muestra una correlación negativa con la calidad.
   d) Los sulfatos tienen una correlación positiva moderada con la calidad.

3. Visualización: Se creó un gráfico de la distribución de calidad ('wine_quality_distribution.png').

Este análisis proporciona insights sobre los factores que influyen en la calidad del vino tinto,
útiles para productores y entusiastas del vino.
""")

spark.stop()

Total de muestras: 1599
+-------+-----+
|quality|count|
+-------+-----+
|      3|   10|
|      4|   53|
|      5|  681|
|      6|  638|
|      7|  199|
|      8|   18|
+-------+-----+

Top 5 vinos con mayor contenido de alcohol:
+-------+-------+----+
|alcohol|quality|rank|
+-------+-------+----+
|   14.9|      5|   1|
|   14.0|      6|   2|
|   14.0|      6|   2|
|   14.0|      6|   2|
|   14.0|      8|   2|
|   14.0|      7|   2|
|   14.0|      8|   2|
|   14.0|      6|   2|
+-------+-------+----+

Distribución por categoría de calidad:


+----------------+-----+
|quality_category|count|
+----------------+-----+
|           Medio| 1319|
|            Alto|  217|
|            Bajo|   63|
+----------------+-----+

Correlación entre fixed_acidity y calidad: 0.124
Correlación entre volatile_acidity y calidad: -0.391
Correlación entre citric_acid y calidad: 0.226
Correlación entre residual_sugar y calidad: 0.014
Correlación entre chlorides y calidad: -0.129
Correlación entre free_sulfur_dioxide y calidad: -0.051
Correlación entre total_sulfur_dioxide y calidad: -0.185
Correlación entre density y calidad: -0.175
Correlación entre pH y calidad: -0.058
Correlación entre sulphates y calidad: 0.251
Correlación entre alcohol y calidad: 0.476

Informe de Análisis de Calidad del Vino Tinto

1. Datos: Se utilizó el conjunto de datos de Calidad del Vino Tinto del repositorio UCI.

2. Hallazgos principales:
   a) La mayoría de los vinos tienen una calidad entre 5 y 6.
   b) Existe una correlación positiva entre el contenido de alcohol y

In [2]:
import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, udf, rank, when
from pyspark.sql.types import StringType
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import io
import logging

# Crear una sesión de Spark con configuración de log reducida
spark = SparkSession.builder \
    .appName("Abalone Analysis") \
    .getOrCreate()

# Reducir el nivel de log de Spark
spark.sparkContext.setLogLevel("ERROR")

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data"
column_names = ["Sex", "Length", "Diameter", "Height", "Whole_weight", "Shucked_weight", "Viscera_weight", "Shell_weight", "Rings"]
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), header=None, names=column_names)
df = spark.createDataFrame(data)

print(f"Total de muestras: {df.count()}")
df.groupBy("Sex").count().orderBy("Sex").show()

# Window Function: Ranking de abulones por peso total
window_spec = Window.orderBy(col("Whole_weight").desc())
ranked_abalones = df.withColumn("rank", rank().over(window_spec))
print("Top 5 abulones más pesados:")
ranked_abalones.select("Whole_weight", "Sex", "Rings", "rank").filter(col("rank") <= 5).show()

# UDF: Categorizar edad
@udf(returnType=StringType())
def categorize_age(rings):
    return "Joven" if rings <= 8 else "Maduro" if rings <= 15 else "Viejo"

df_with_category = df.withColumn("age_category", categorize_age(col("Rings")))
print("Distribución por categoría de edad:")
df_with_category.groupBy("age_category").count().orderBy(col("count").desc()).show()

# Análisis de correlaciones
numeric_features = ["Length", "Diameter", "Height", "Whole_weight", "Shucked_weight", "Viscera_weight", "Shell_weight"]
for feature in numeric_features:
    correlation = df.stat.corr(feature, "Rings")
    print(f"Correlación entre {feature} y edad (anillos): {correlation:.3f}")

# Análisis adicional: Promedio de anillos por sexo
avg_rings_by_sex = df.groupBy("Sex").agg(avg("Rings").alias("avg_rings")).orderBy("avg_rings", ascending=False)
print("Promedio de anillos (edad) por sexo:")
avg_rings_by_sex.show()

# Visualización
plt.figure(figsize=(10, 6))
sex_counts = df.groupBy("Sex").count().orderBy("Sex").collect()
plt.bar([row['Sex'] for row in sex_counts], [row['count'] for row in sex_counts])
plt.title("Distribución de Abulones por Sexo")
plt.xlabel("Sexo")
plt.ylabel("Número de Muestras")
plt.savefig("abalone_sex_distribution.png")
plt.close()

print("""
Informe de Análisis de Abulones

1. Datos: Se utilizó el conjunto de datos de Abulones del repositorio UCI.

2. Hallazgos principales:
   a) La distribución de abulones por sexo muestra diferencias entre machos, hembras e infantes.
   b) Existe una fuerte correlación positiva entre el peso de la concha y la edad (número de anillos).
   c) El diámetro y la longitud también están correlacionados positivamente con la edad.
   d) Los abulones más pesados tienden a ser los más viejos, pero hay excepciones.

3. Visualización: Se creó un gráfico de la distribución de abulones por sexo ('abalone_sex_distribution.png').

Este análisis proporciona insights sobre las características físicas de los abulones y cómo se relacionan con su edad,
útiles para biólogos marinos y la industria pesquera.
""")

spark.stop()

Total de muestras: 4177
+---+-----+
|Sex|count|
+---+-----+
|  F| 1307|
|  I| 1342|
|  M| 1528|
+---+-----+

Top 5 abulones más pesados:
+------------+---+-----+----+
|Whole_weight|Sex|Rings|rank|
+------------+---+-----+----+
|      2.8255|  M|   17|   1|
|      2.7795|  M|   12|   2|
|       2.657|  F|   11|   3|
|       2.555|  F|   11|   4|
|        2.55|  M|   14|   5|
+------------+---+-----+----+

Distribución por categoría de edad:


+------------+-----+
|age_category|count|
+------------+-----+
|      Maduro| 2509|
|       Joven| 1407|
|       Viejo|  261|
+------------+-----+

Correlación entre Length y edad (anillos): 0.557
Correlación entre Diameter y edad (anillos): 0.575
Correlación entre Height y edad (anillos): 0.557
Correlación entre Whole_weight y edad (anillos): 0.540
Correlación entre Shucked_weight y edad (anillos): 0.421
Correlación entre Viscera_weight y edad (anillos): 0.504
Correlación entre Shell_weight y edad (anillos): 0.628
Promedio de anillos (edad) por sexo:
+---+------------------+
|Sex|         avg_rings|
+---+------------------+
|  F|11.129303749043611|
|  M|10.705497382198953|
|  I| 7.890461997019374|
+---+------------------+


Informe de Análisis de Abulones

1. Datos: Se utilizó el conjunto de datos de Abulones del repositorio UCI.

2. Hallazgos principales:
   a) La distribución de abulones por sexo muestra diferencias entre machos, hembras e infantes.
   b) Existe una fuerte correlaci